In [1]:
import tensorflow as tf
import tensorflow.keras as keras
tf.__version__

'2.0.0-rc2'

### 卷积
卷积和互相关计算是等价的，所以使用互相关代替卷积计算

In [6]:
def corr2D(X,K):
    h,w = K.shape
    out = []
    for i in range(X.shape[0]-h+1):
        for j in range(X.shape[1] - w +1):
            out.append(tf.reduce_sum(tf.multiply(X[i:i+h,j:j+w],K)))
    
    return tf.reshape(tf.stack(out),shape=(X.shape[0]-h+1,X.shape[1] - w +1))

In [3]:
x = tf.random.normal(shape=(5,5))
x

<tf.Tensor: id=5, shape=(5, 5), dtype=float32, numpy=
array([[ 0.22155085, -0.40791768,  0.8558023 , -0.4797462 , -1.0513757 ],
       [ 0.679054  , -1.2940665 ,  0.57222134,  0.6770314 ,  0.90435135],
       [-0.5671288 , -0.38408816, -0.5437065 , -1.1441692 ,  0.26477835],
       [ 1.3730546 , -1.9406254 , -0.76116943, -1.0714163 ,  0.03664621],
       [-0.04350886,  0.10470191, -0.17594865, -0.0635838 ,  0.04465179]],
      dtype=float32)>

In [4]:
k = tf.random.normal(shape=(3,3))
k

<tf.Tensor: id=11, shape=(3, 3), dtype=float32, numpy=
array([[-0.10839651, -1.2529025 , -1.0445781 ],
       [-1.1433839 , -0.62975216,  2.026606  ],
       [ 0.1896122 , -0.42764854,  0.4994575 ]], dtype=float32)>

In [7]:
y = corr2D(x,k)
y

<tf.Tensor: id=77, shape=(3, 3), dtype=float32, numpy=
array([[ 0.5764644 ,  1.552669  ,  2.877141  ],
       [ 1.4485314 , -3.398677  ,  0.3560288 ],
       [-0.92066675,  2.5082479 ,  2.8513207 ]], dtype=float32)>

### 多通道输入
多通道输入时建立和输入通道数相同的卷积核，输出是各个通道互相关计算后的和

In [18]:
def corr2D_mutil_in(X,K):
    out = []
    for x,k in zip(X,K):
        out.append(corr2D(x,k))
    
    y = tf.stack(out)
    
    return tf.reduce_sum(y,axis=0)

In [10]:
x = tf.random.normal(shape=(3,8,8))
x

<tf.Tensor: id=83, shape=(3, 8, 8), dtype=float32, numpy=
array([[[ 0.63328826, -0.21243227, -0.82599133, -1.1887453 ,
         -1.045694  , -0.2902387 ,  0.57050604, -0.17830798],
        [ 1.0887088 , -0.57618695, -2.5827413 ,  2.4158032 ,
         -0.3870853 ,  0.6673154 , -0.51990205,  0.15159066],
        [-1.2004771 ,  1.3980432 , -0.25103816, -0.09256177,
         -2.2871041 , -0.01998747, -0.44037914,  0.61005014],
        [ 1.0967577 , -1.4816341 , -1.3349471 , -1.6829419 ,
         -0.09218091, -1.0556115 ,  0.42174593,  0.94709754],
        [-0.47990704,  0.95362175,  1.3351692 ,  0.97789955,
         -0.6550805 , -0.83845586, -0.02976377, -2.7110348 ],
        [-0.05819235, -0.1428613 , -1.6522063 , -2.6383848 ,
          0.3253129 , -0.02537183,  1.2855124 ,  0.21517548],
        [ 1.1909647 ,  1.3558509 , -1.5991309 , -0.75305307,
         -0.9125243 ,  0.11075728,  0.6995201 ,  1.3229028 ],
        [-0.62761337, -0.15930445,  2.0590727 , -0.8728582 ,
         -1.0155467 

In [11]:
k = tf.random.normal(shape=(3,3,3))
k

<tf.Tensor: id=89, shape=(3, 3, 3), dtype=float32, numpy=
array([[[-1.270996  ,  1.4949868 , -0.3312518 ],
        [-0.70502627, -0.16828637, -1.8107255 ],
        [-1.6423343 ,  0.7182905 ,  0.7223735 ]],

       [[ 0.48632306,  0.12262274, -1.70743   ],
        [-1.0931759 ,  0.48210412, -0.4297394 ],
        [ 0.01149237,  0.10243513, -0.40166318]],

       [[ 0.97528607, -0.7184189 ,  0.5290594 ],
        [-0.29424953, -1.6980512 ,  1.3347925 ],
        [-0.7718224 , -0.45587268,  0.14089411]]], dtype=float32)>

In [19]:
corr2D_mutil_in(x,k)

<tf.Tensor: id=2468, shape=(6, 6), dtype=float32, numpy=
array([[ 13.275757  , -10.09803   ,   0.83173126, -11.254443  ,
         10.355454  ,  -2.7743037 ],
       [ -2.026022  ,  -6.012504  ,  11.032714  ,   0.14585078,
          1.065505  ,   6.919719  ],
       [  6.2903557 ,  -4.7215595 ,   3.0539308 ,  -4.2785144 ,
          5.9604397 ,  -0.1661278 ],
       [ -7.639618  ,  -6.514247  ,   2.533553  ,   7.291796  ,
          4.8334837 ,   3.1606498 ],
       [  1.6064616 ,   0.1537137 ,   2.7672853 ,   2.6140285 ,
          5.4339724 ,  -1.3248696 ],
       [  2.5194795 ,   2.0549297 ,  -3.9770508 ,   3.9968271 ,
          6.193736  ,  -8.309581  ]], dtype=float32)>

### 多通道输出
多通道输出时，建立和输出通道个数相同的卷积和，如果输入通道数是Ci，输出通道数是Co，卷积核大小是Kh，Kw，那么卷积和的参数有Ci × Co × Kh × Kw

In [22]:
def mutil_in_out(X,K):
    '''Co->Ci->Kh->Kw'''
    arr_k = tf.unstack(k)
    out = []
    for i in arr_k:
        out.append(corr2D_mutil_in(X,i))
    return tf.stack(out)

In [23]:
## 输入三个通道，大小是8x8
x = tf.random.normal(shape=(3,8,8))

In [24]:
#输出5个通道,输入三个通道，大小是3x3
k = tf.random.normal(shape=(5,3,3,3))

In [25]:
mutil_in_out(x,k)

<tf.Tensor: id=6455, shape=(5, 6, 6), dtype=float32, numpy=
array([[[  9.2593975 ,  -0.4723799 ,   7.385053  ,  -5.348192  ,
           0.20169479,  -5.5525217 ],
        [  8.814385  ,  -1.0497329 ,   2.2275696 ,  -3.2593899 ,
           2.6276689 ,   8.079218  ],
        [  4.82032   ,  -0.6243248 ,  -1.6277196 ,   4.331703  ,
          10.867795  ,   2.6935534 ],
        [  0.6841079 ,  -1.0424513 ,   0.52341413,   1.5952902 ,
           8.187709  ,  12.0358095 ],
        [  2.2600102 ,   4.1235237 ,   3.162024  ,   5.700164  ,
          -5.314061  ,   5.668768  ],
        [  3.3228445 ,  -3.7297423 ,   3.158123  ,   0.3302741 ,
          -1.4993205 ,  -5.510884  ]],

       [[ -4.415243  ,   0.23474312,  -0.3406942 ,   3.715579  ,
           5.3310556 ,  -6.0403395 ],
        [ -4.8318553 ,  10.827856  ,  -0.24144244,   9.7760725 ,
          -2.0725422 ,   0.3567953 ],
        [  4.2437716 ,   7.543081  ,  -9.20323   ,  -2.8866503 ,
          -9.304418  ,   3.2807012 ],
        [ -